<a href="https://colab.research.google.com/github/shalinisaini/CS60092_9A_18CS60R47/blob/master/DL_Assign.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
SEED =42
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)

import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 


In [242]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [0]:

url = 'https://raw.githubusercontent.com/binny-mathew/IITKGP_CS69002_Spring_2019/master/Dataset/Test_5K.csv'
df1 = pd.read_csv(url, sep='\t')

In [0]:
url = 'https://raw.githubusercontent.com/binny-mathew/IITKGP_CS69002_Spring_2019/master/Dataset/Train_20K.csv'
df2 = pd.read_csv(url, sep='\t')

In [0]:

train_data = df2['text'].astype(str).tolist()
train_label = df2['label'].astype(int)
test_data = df1['text'].astype(str).tolist()
test_label = df1['label'].astype(int)

train_data = [x.lower() for x in train_data]
test_data = [x.lower() for x in test_data]


 


In [0]:
def preprocess(data):
  stopword = set(stopwords.words('english'))
  dt2=[]
  for i in data:
    words = i.split(" ")
    dt =[]
    for j in words:
      if(j not in stopword and not j.isdigit() and len(j)>1):
        dt.append(j)
    dt2.append(dt)
  return dt2

              

In [0]:

train_data = preprocess(train_data)
test_data = preprocess(test_data)

In [248]:
print(len(train_data))

17999


In [0]:
def generate_word_ids(dataset):
  word_to_ix = {}
  print(dataset)
  for sent in dataset:
      for word in sent:
          if word not in word_to_ix:
              word_to_ix[word] = 1
          else:
              word_to_ix[word] = word_to_ix[word]+1
  return word_to_ix

In [250]:
word_to_ix = generate_word_ids(train_data)
print(len(word_to_ix))
NUM_LABELS = 2



IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



204915


In [251]:
list=[]
for i in word_to_ix:
  if(word_to_ix[i]>10):
    list.append(i)
word_to_ix={}
for i in list:
  word_to_ix[i] = len(word_to_ix)
  
VOCAB_SIZE = len(word_to_ix)
print(VOCAB_SIZE)

19704


In [0]:
class BOWClassifier(nn.Module):
  def __init__(self, num_labels, vocab_size):
    super(BOWClassifier, self).__init__()
    self.dp=nn.Dropout(0.2)
    self.dp1=nn.Dropout(0.3)
    self.dp2=nn.Dropout(0.4)
    self.lin = nn.Linear(vocab_size, 30)
    self.lin4 = nn.Linear(30,20)
    self.lin5 = nn.Linear(20,10)
    self.lin6 = nn.Linear(10, num_labels)
    
    
    
  def forward(self, x):
    x=F.relu(self.lin(x))
    x=self.dp(x)
    x=F.relu(self.lin4(x))
    x=self.dp1(x)
    x=F.relu(self.lin5(x))
    x=self.dp2(x)
    return F.softmax(self.lin6(x))
  
  
  #task1a ---
    #class BOWClassifier(nn.Module):
    #def __init__(self, num_labels, vocab_size):
    #super(BOWClassifier, self).__init__()
    #self.lin = nn.Linear(vocab_size, 50)
    #self.lin2 = nn.Linear(50, num_labels)
    #def forward(self, x):
     #return F.softmax(self.lin2(self.lin(x)))
    
    
  #task1b ---
    #class BOWClassifier(nn.Module):
    #def __init__(self, num_labels, vocab_size):
    #super(BOWClassifier, self).__init__()
    #self.lin = nn.Linear(vocab_size, 100)
    #self.lin2 = nn.Linear(100, num_labels)
    #def forward(self, x):
      #return F.softmax(self.lin2(self.lin(x)))
    
    
  #task1c ---
    #class BOWClassifier(nn.Module):
    #def __init__(self, num_labels, vocab_size):
    #super(BOWClassifier, self).__init__()
    #self.lin = nn.Linear(vocab_size, 200)
    #self.lin2 = nn.Linear(200, num_labels)
    #def forward(self, x):
      #return F.softmax(self.lin2(self.lin(x)))
    
    
  #task1d ---
    #class BOWClassifier(nn.Module):
    #def __init__(self, num_labels, vocab_size):
    #super(BOWClassifier, self).__init__()
    #self.lin = nn.Linear(vocab_size, 500)
    #self.lin2 = nn.Linear(500, num_labels)
     #def forward(self, x):
      #return F.softmax(self.lin2(self.lin(x)))
  
 
    
  #task2a ---
    #class BOWClassifier(nn.Module):
    #def __init__(self, num_labels, vocab_size):
    #super(BOWClassifier, self).__init__()
    #self.lin = nn.Linear(vocab_size, 10)
    #self.lin2 = nn.Linear(10, 10)
    #self.lin3 = nn.Linear(10, num_labels)
    #def forward(self, x):
     #return F.softmax(self.lin3(self.lin2(self.lin(x))))
  
  #task2b ---
    #class BOWClassifier(nn.Module):
    #def __init__(self, num_labels, vocab_size):
    #super(BOWClassifier, self).__init__()
    #self.lin = nn.Linear(vocab_size, 20)
    #self.lin2 = nn.Linear(20, 10)
    #self.lin3 = nn.Linear(10, num_labels)
    #def forward(self, x):
      #return F.softmax(self.lin3(self.lin2(self.lin(x))))
    
   #task2c ---
    #class BOWClassifier(nn.Module):
    #def __init__(self, num_labels, vocab_size):
    #super(BOWClassifier, self).__init__()
    #self.lin = nn.Linear(vocab_size, 30)
    #self.lin2 = nn.Linear(30, 30)
    #self.lin3 = nn.Linear(30, num_labels)
    #def forward(self, x):
     #return F.softmax(self.lin3(self.lin2(self.lin(x))))
    
   #task2d ---
    #class BOWClassifier(nn.Module):
    #def __init__(self, num_labels, vocab_size):
    #super(BOWClassifier, self).__init__()
    #self.lin = nn.Linear(vocab_size, 50)
    #self.lin2 = nn.Linear(50, 50)
    #self.lin3 = nn.Linear(50, num_labels)
    #def forward(self, x):
      #return F.softmax(self.lin3(self.lin2(self.lin(x))))
    
   #task2e ---
    #class BOWClassifier(nn.Module):
    #def __init__(self, num_labels, vocab_size):
    #super(BOWClassifier, self).__init__()
    #self.lin = nn.Linear(vocab_size, 100)
    #self.lin2 = nn.Linear(100, 50)
    #self.lin3 = nn.Linear(50, num_labels)
    #def forward(self, x):
     #return F.softmax(self.lin3(self.lin2(self.lin(x))))
    
  #task3a----
    #class BOWClassifier(nn.Module):
    #def __init__(self, num_labels, vocab_size):
    #super(BOWClassifier, self).__init__()
    #self.lin = nn.Linear(vocab_size, 100)
    #self.lin4 = nn.Linear(100,50)
    #self.lin5 = nn.Linear(50,10)
    #self.lin6 = nn.Linear(10, num_labels)
    #def forward(self, x):
      #return F.softmax(self.lin6(self.lin5(self.lin4(self.lin(x)))))
  
  
   #task3b-----  
    #class BOWClassifier(nn.Module):
    #def __init__(self, num_labels, vocab_size):
    #super(BOWClassifier, self).__init__()
    #self.lin = nn.Linear(vocab_size, 200)
    #self.lin4 = nn.Linear(200,100)
    #self.lin5 = nn.Linear(100,10)
    #self.lin6 = nn.Linear(10, num_labels)
    
   #def forward(self, x):
    #return F.softmax(self.lin6(self.lin5(self.lin4(self.lin(x)))))
  
  
  #task 4a----
    #class BOWClassifier(nn.Module):
    #def __init__(self, num_labels, vocab_size):
    #super(BOWClassifier, self).__init__()
    #self.dp=nn.Dropout(0.4)
    #self.lin = nn.Linear(vocab_size, 30)
    #self.lin4 = nn.Linear(30,20)
    #self.lin5 = nn.Linear(20,10)
    #self.lin6 = nn.Linear(10, num_labels)
    #def forward(self, x):
    #x=self.lin(x)
    #x=self.dp(x)
    #x=self.lin4(x)
    #x=self.dp(x)
    #x=self.lin5(x)
    #x=self.dp(x)
    #return F.softmax(self.lin6(x))
  
    
  
 #task4b----
    #class BOWClassifier(nn.Module):
    #def __init__(self, num_labels, vocab_size):
    #super(BOWClassifier, self).__init__()
    #self.dp=nn.Dropout(0.1)
    #self.dp1=nn.Dropout(0.2)
    #self.lin = nn.Linear(vocab_size, 100)
    #self.lin4 = nn.Linear(100,100)
    #self.lin6 = nn.Linear(100, num_labels)
    #def forward(self, x):
    #x=F.relu(self.lin(x))
    #x=self.dp(x)
    #x=F.relu(self.lin4(x))
    #x=self.dp1(x)
    #return F.softmax(self.lin6(x))
  
   
 #task4c----
    #class BOWClassifier(nn.Module):
    #def __init__(self, num_labels, vocab_size):
    #super(BOWClassifier, self).__init__()
    #self.dp=nn.Dropout(0.1)
    #self.dp1=nn.Dropout(0.2)
    #self.lin = nn.Linear(vocab_size, 100)
    #self.lin4 = nn.Linear(100,10)
    #self.lin6 = nn.Linear(10, num_labels)
    #def forward(self, x):
    #x=F.relu(self.lin(x))
    #x=self.dp(x)
    #x=F.relu(self.lin4(x))
    #x=self.dp1(x)
    #return F.softmax(self.lin6(x))
  
    

In [0]:
def make_bow_vector(sentence, word_to_ix):
    # create a vector of zeros of vocab size = len(word_to_idx)
    vec = torch.zeros(len(word_to_ix))
    for word in sentence:
        if word not in word_to_ix:
            #raise ValueError('Word',word,' not present in the dictionary. Sorry!')
            c=1
        else:
            vec[word_to_ix[word]]+=1
    return vec.view(1, -1)

def make_target(label, label_to_ix):
    return torch.LongTensor([label_to_ix[label]])


In [0]:
bow = BOWClassifier(NUM_LABELS, VOCAB_SIZE).cuda()


In [0]:
def accuracy(data,label):
  count=0
  tp =0
  tn=0
  fp=0
  fn=0
  for idx in range(len(data)):
      bow_vec = Variable(make_bow_vector(data[idx], word_to_ix)).cuda()
      labels = Variable(torch.LongTensor(label[idx])).cuda()
      logprobs = bow(bow_vec).reshape(-1)
      val,pred = logprobs.max(0)
      #print(pred,label[idx]+1)
      if(pred == label[idx] and pred==1):
        count+=1
        tp+=1
        
      elif(pred == label[idx] and pred==0):
        tn+=1
      
      elif(pred !=label[idx] and pred==1):
        fp+=1
        
      elif(pred !=label[idx] and pred==0):
        fn+=1
  
  #precision =(float(tp)/(tp+fp))
  #recall=(float(tp/(tp+fn)))
  #print("precision {}".format(float(tp)/(tp+fp)))
  #print("recall {}".format(float(tp/(tp+fn))))
  #print("F1 Score {}".format(float(2*((precision*recall)/(precision+recall)))))
  print("accuracy {}".format(float(count/len(data))))
        


In [0]:
# define a loss function and an optimizer
loss_function = nn.CrossEntropyLoss()
opt = torch.optim.SGD(bow.parameters(), lr = 0.1)

In [0]:
# the training loop
for epoch in range(5):
    print("epoch",epoch)
    for idx in range(len(train_data)):
        # get the training data
        bow.zero_grad()
        #print(train_data[idx])
        #print(train_label[idx])
        bow_vec = Variable(make_bow_vector(train_data[idx], word_to_ix)).cuda()
        label  = Variable(torch.LongTensor([train_label[idx]])).cuda()
        #print(bow_vec,label,train_label[idx])
        probs = bow(bow_vec) # forward pass
        #print(probs)
        loss = loss_function(probs, label)
        loss.backward()
        opt.step()
    print('CURRENT LOSS: {}'.format(loss.data))
    accuracy(train_data,train_label)


In [264]:
accuracy(test_data,test_label)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


precision 0.503500700140028
recall 0.9996028594122319
F1 Score 0.6696820540109085
accuracy 0.5034


In [0]:
torch.save(bow.cpu(),'Task5A.mdl')
from google.colab import files
files.download("Task5A.mdl")

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type BOWClassifier. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
